In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
import pickle

from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, LayerNormalization, Conv2D, MaxPooling2D, Input, MultiHeadAttention, Layer, Flatten, Reshape, Concatenate
from tensorflow.keras.optimizers import Adam

import numpy as np


# Custom Transformer Block
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([Dense(ff_dim, activation='gelu'), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Sample data
text = "Hello world. This is a sample text for language modeling."

# Data augmentation: Adding more variations of the sample text
augmented_texts = [
    "Hello world! This is a sample text for language modeling.",
    "Hi world. This is an example text for modeling languages.",
    "Greetings, world. This text serves as a sample for language modeling.",
    "Hey world! Here's a sample text for language modeling purposes.",
    "Hello universe. This is a test text for language modeling.",
]

# Combine original and augmented texts
all_texts = [text] + augmented_texts

# Tokenize the text
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)

# Save the tokenizer to a file in Google Drive
tokenizer_path = '/content/drive/MyDrive/converted_models/tokenizer.pickle'
with open(tokenizer_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Prepare the dataset
seq_length = 10  # Number of previous tokens to consider

# Generate sequences
dataset = []
for seq in sequences:
    for i in range(seq_length, len(seq)):
        dataset.append(seq[i-seq_length:i])

dataset = np.array(dataset)

if dataset.size == 0:
    raise ValueError("The dataset is empty. Please check the sequence length and the text data.")

X, y = dataset[:, :-1], dataset[:, -1]

# Ensure X has shape (None, 9)
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Adjust input shape to match the expected shape
inputs = Input(shape=(seq_length-1,))  # seq_length-1 because we're predicting the next token
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=seq_length-1)(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)(embedding_layer)

# Debug: Print shape after transformer block
print(f"Shape after transformer block: {transformer_block.shape}")

reshape_layer = Reshape((seq_length-1, embed_dim, 1))(transformer_block)

# Debug: Print shape after reshape for Conv2D
print(f"Shape after reshape for Conv2D: {reshape_layer.shape}")

conv_layer = Conv2D(64, (3, 3), activation='relu', padding='same')(reshape_layer)
pool_layer = MaxPooling2D(pool_size=(2, 2))(conv_layer)
flatten_layer = Flatten()(pool_layer)

# Debug: Print shape after flattening
print(f"Shape after flattening: {flatten_layer.shape}")

# Adjust the reshaping to ensure correct input shape for LSTM
new_shape = (flatten_layer.shape[1] // embed_dim, embed_dim)
lstm_input = Reshape(new_shape)(flatten_layer)

# Debug: Print shape before LSTM
print(f"Shape before LSTM: {lstm_input.shape}")

lstm_layer = LSTM(lstm_units, return_sequences=True)(lstm_input)

# Debug: Print shape after first LSTM layer
print(f"Shape after first LSTM layer: {lstm_layer.shape}")

lstm_layer = LSTM(50)(lstm_layer)

# Debug: Print shape after second LSTM layer
print(f"Shape after second LSTM layer: {lstm_layer.shape}")

concat_layer = Concatenate()([flatten_layer, lstm_layer])
outputs = Dense(vocab_size, activation='softmax')(concat_layer)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=2)  # Increase epochs for better training

# Save the trained model
model.save('/content/drive/MyDrive/converted_models/HINAI_upgraded_with_transformer.h5')

# Load the trained model and tokenizer for text generation
model = load_model('/content/drive/MyDrive/converted_models/HINAI_upgraded_with_transformer.h5', custom_objects={'TransformerBlock': TransformerBlock})

with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)

def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        if len(token_list) > seq_length - 1:
            token_list = token_list[-(seq_length-1):]  # Truncate sequences longer than seq_length-1
        token_list = np.pad(token_list, (seq_length-1-len(token_list), 0), 'constant')
        token_list = np.array([token_list])
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1)[0]
        output_word = tokenizer.index_word.get(predicted_word_index, '<OOV>')
        seed_text += " " + output_word
    return seed_text

# Test the model
seed_text = "Hello world"
generated_text = generate_text(seed_text, next_words=20)
print(generated_text)
"""

# Hyperparameter tuning (manual approach)
for embed_dim in [64, 128]:
    for num_heads in [4, 8]:
        for ff_dim in [64, 128]:
            for lstm_units in [50, 100]:
                print(f"Training with embed_dim={embed_dim}, num_heads={num_heads}, ff_dim={ff_dim}, lstm_units={lstm_units}")
                inputs = Input(shape=(seq_length-1,))
                embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=seq_length-1)(inputs)
                transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)(embedding_layer)
                reshape_layer = Reshape((seq_length-1, embed_dim, 1))(transformer_block)
                conv_layer = Conv2D(64, (3, 3), activation='relu', padding='same')(reshape_layer)
                pool_layer = MaxPooling2D(pool_size=(2, 2))(conv_layer)
                flatten_layer = Flatten()(pool_layer)
                lstm_input = Reshape((flatten_layer.shape[1] // embed_dim, embed_dim))(flatten_layer)
                lstm_layer = LST
                """



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape of X: (1, 9)
Shape of y: (1,)
Shape after transformer block: (None, 9, 64)
Shape after reshape for Conv2D: (None, 9, 64, 1)
Shape after flattening: (None, 8192)
Shape before LSTM: (None, 128, 64)
Shape after first LSTM layer: (None, 128, 100)
Shape after second LSTM layer: (None, 50)
Epoch 1/50
1/1 - 6s - loss: 3.2397 - accuracy: 0.0000e+00 - 6s/epoch - 6s/step
Epoch 2/50
1/1 - 0s - loss: 2.2332 - accuracy: 1.0000 - 20ms/epoch - 20ms/step
Epoch 3/50
1/1 - 0s - loss: 1.4218 - accuracy: 1.0000 - 21ms/epoch - 21ms/step
Epoch 4/50
1/1 - 0s - loss: 0.7734 - accuracy: 1.0000 - 20ms/epoch - 20ms/step
Epoch 5/50
1/1 - 0s - loss: 0.4083 - accuracy: 1.0000 - 18ms/epoch - 18ms/step
Epoch 6/50
1/1 - 0s - loss: 0.1682 - accuracy: 1.0000 - 20ms/epoch - 20ms/step
Epoch 7/50
1/1 - 0s - loss: 0.0723 - accuracy: 1.0000 - 19ms/epoch - 19ms/step
Epoch 8/50
1/1 - 0s - loss:

'\n\n# Hyperparameter tuning (manual approach)\nfor embed_dim in [64, 128]:\n    for num_heads in [4, 8]:\n        for ff_dim in [64, 128]:\n            for lstm_units in [50, 100]:\n                print(f"Training with embed_dim={embed_dim}, num_heads={num_heads}, ff_dim={ff_dim}, lstm_units={lstm_units}")\n                inputs = Input(shape=(seq_length-1,))\n                embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim, input_length=seq_length-1)(inputs)\n                transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)(embedding_layer)\n                reshape_layer = Reshape((seq_length-1, embed_dim, 1))(transformer_block)\n                conv_layer = Conv2D(64, (3, 3), activation=\'relu\', padding=\'same\')(reshape_layer)\n                pool_layer = MaxPooling2D(pool_size=(2, 2))(conv_layer)\n                flatten_layer = Flatten()(pool_layer)\n                lstm_input = Reshape((flatten_layer.shape[1] // embed_dim, embed_dim))(f